In [1]:
from __future__ import annotations

from etils.lazy_imports import *
import notion_client

ecolab.clear_cached_modules(['common'])
import common

In [8]:
common.add_notion_to_sys_path()

In [169]:
ecolab.clear_cached_modules(['auto_notion'])

import auto_notion

In [170]:
db = auto_notion.Database('989b65ec51244c8ba318a982717b085d')
db

In [177]:

# Shift TODAY from 7 hours (as 2am on Friday is still Thursday)
TODAY_DATE = datetime.datetime.today() - datetime.timedelta(hours=6)


def next_weekday(curr_day: datetime.datetime, day_id: int):
    day_ahead = (day_id - curr_day.weekday()) % 7
    if day_ahead == 0:
        day_ahead += 7
    return curr_day + datetime.timedelta(days=day_ahead)


def next_monthday(curr_day: datetime.datetime, month_id: int):
    if month_id > curr_day.month:  # Keep current year
        year = curr_day.year
    else:  # Month id already passed. Use next year
        year = curr_day.year + 1
    return datetime.datetime(year=year, month=month_id, day=1)


@dataclasses.dataclass
class NextWeek:
    day_id: int

    def __call__(self) -> Any:
        return next_weekday(TODAY_DATE, self.day_id)


@dataclasses.dataclass
class DetlaTime:
    days: int

    def __call__(self) -> Any:
        return TODAY_DATE + datetime.timedelta(days=self.days)


@dataclasses.dataclass
class InMonth:
    month_id: int

    def __call__(self) -> Any:
        return next_monthday(TODAY_DATE, self.month_id)


option_to_delta = {
    'Next Monday': NextWeek(0),
    'Next Tuesday': NextWeek(1),
    'Next Wednesday': NextWeek(2),
    'Next Thursday': NextWeek(3),
    'Next Friday': NextWeek(4),
    'Next Saturday': NextWeek(5),
    'Next Sunday': NextWeek(6),
    'In 1 Day': DetlaTime(1),
    'In 7 Days': DetlaTime(7),
    'In 1 Month': DetlaTime(30),
    'In 2 Months': DetlaTime(60),
    'In 1 Year': DetlaTime(365),
    'In Jan': InMonth(1),
    'In Feb': InMonth(2),
    'In Mar': InMonth(3),
    'In Apr': InMonth(4),
    'In May': InMonth(5),
    'In Jun': InMonth(6),
    'In Jul': InMonth(7),
    'In Aug': InMonth(8),
    'In Sep': InMonth(9),
    'In Oct': InMonth(10),
    'In Nov': InMonth(11),
    'In Dec': InMonth(12),
    'In the US': None,
}

In [173]:
all_exs = list(db[~db.filters.done])

Unsupported property date
Unsupported property select
Unsupported property created_time
Unsupported property select
Unsupported property date
Unsupported property title


In [181]:
print('Now: ', TODAY_DATE)

for row in all_exs:
    if not row.snooze:
        continue
    if row.snooze not in option_to_delta:
        raise ValueError(f'Unexpected snooze value: {row.snooze!r}')

    delta_fn = option_to_delta[row.snooze]
    if delta_fn is None:
        continue

    reminder = delta_fn().date()
    print(f'{row.snooze} -> {reminder}')
    row.reminder = reminder
    row.snooze = None

Now:  2023-01-14 15:39:34.011682
In Feb -> 2024-01-01
Next Saturday -> 2023-01-21
Next Monday -> 2023-01-16
Next Saturday -> 2023-01-21
Next Saturday -> 2023-01-21
In Apr -> 2023-03-01
Next Tuesday -> 2023-01-17
In 1 Month -> 2023-02-13
In 7 Days -> 2023-01-21
In Apr -> 2023-03-01
In Feb -> 2024-01-01
In 1 Month -> 2023-02-13
Next Sunday -> 2023-01-15
In 2 Months -> 2023-03-15
In 1 Month -> 2023-02-13
In Mar -> 2023-02-01
In 1 Day -> 2023-01-15
In 1 Year -> 2024-01-14


In [136]:
TODAY_DATE = datetime.datetime.today()

for row in db[db.filters.done]:
    if not row.archived:
        row.archived = TODAY_DATE

Unsupported property date
Unsupported property select
Unsupported property created_time
Unsupported property select
Unsupported property date
Unsupported property title
